In [22]:
import numpy as np
import pandas as pd

from scipy.sparse import load_npz
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

In [23]:
import gc
gc.collect()

3272

In [24]:
input_base_path = '/media/hiroki/working/kaggle_data/trends-neuroimaging/split_IC'
out_path = '/media/hiroki/working/kaggle_data/trends-neuroimaging/split_IC/SVD'

In [25]:
ic1_train = load_npz(input_base_path + '/train/IC/ic1_matrix.npz')
ic1_test = load_npz(input_base_path + '/test/IC/ic1_matrix.npz')

In [26]:
ic1 = sparse.vstack([ic1_train, ic1_test])

In [27]:
del ic1_train, ic1_test

In [28]:
ic1

<11754x173628 sparse matrix of type '<class 'numpy.float64'>'
	with 691946226 stored elements in Compressed Sparse Row format>

In [29]:
%%time
svd = TruncatedSVD(n_components=30, random_state=42)
decomp_ic1 = svd.fit_transform(ic1)
del ic1

CPU times: user 4min 11s, sys: 288 ms, total: 4min 11s
Wall time: 4min 10s


CPU times: user 2min 28s, sys: 86.3 ms, total: 2min 28s
Wall time: 2min 27s

In [19]:
svd.explained_variance_ratio_.sum()

array([0.01483676, 0.0163475 , 0.00539418, 0.00363033, 0.00246295,
       0.00207645, 0.00170548, 0.00166786, 0.00157847, 0.00148   ,
       0.00141285, 0.00140765, 0.00130038, 0.00127598, 0.00124614,
       0.00122619, 0.0012035 , 0.00118402, 0.00117571, 0.00114885,
       0.00113265, 0.00110651, 0.00108613, 0.00107692, 0.0010696 ,
       0.00104455, 0.00104243, 0.00102756, 0.00102043, 0.0010147 ,
       0.00100813, 0.00099314, 0.00099061, 0.00097962, 0.00097252,
       0.0009719 , 0.0009678 , 0.00095952, 0.00095862, 0.00094534,
       0.00094489, 0.0009351 , 0.0009306 , 0.00092676, 0.0009216 ,
       0.00091647, 0.00091366, 0.00090796, 0.00090592, 0.0009002 ,
       0.00089533, 0.00089385, 0.00088946, 0.00088601, 0.00088463,
       0.0008823 , 0.00087851, 0.00087677, 0.0008724 , 0.0008674 ,
       0.00086108, 0.00085899, 0.00085479, 0.00085112, 0.00084652,
       0.00084464, 0.0008415 , 0.00083513, 0.0008344 , 0.0008327 ,
       0.00083181, 0.00082527, 0.00082014, 0.00081899, 0.00081

In [17]:
ratio = svd.explained_variance_ratio_

In [20]:
ratio.sum()

0.13426221522496867

In [26]:
np.save(out_path+'svd_ic1_mtx',decomp_ic1)
del decomp_ic1

In [30]:
svd_ic1 = np.load(out_path+'/svd_ic1_mtx.npy')

In [31]:
svd_ic1_df = pd.DataFrame(svd_ic1,columns=['IC1_sv_0','IC1_sv_1','IC1_sv_2'])

In [32]:
svd_ic1_df

,IC1_svd_0,IC1_svd_1,IC1_svd_2
0,171.218424,-6.427769,-9.923038
1,183.244780,-50.385175,7.093901
2,176.161957,-7.524484,-10.002003
3,181.348800,-14.650756,-17.043148
4,180.138250,-5.162861,-9.383822
...,...,...,...
11749,180.950685,-40.158520,-2.057099
11750,176.745390,15.437133,-14.263656
11751,169.971154,12.118586,25.889939
11752,160.253248,11.686251,15.355906


# run all SVD

In [6]:
%%time
svd_ic_dfs = pd.DataFrame()
for num in range(1,54):
    ic_train = load_npz(input_base_path + '/train/IC/ic{}_matrix.npz'.format(num))
    ic_test = load_npz(input_base_path + '/test/IC/ic{}_matrix.npz'.format(num))
    ic = sparse.vstack([ic_train, ic_test])
    del ic_train, ic_test
    
    svd = TruncatedSVD(n_components=3, random_state=42)
    decomp_ic = svd.fit_transform(ic)
    del ic
    if num == 1:
        svd_ic_dfs = pd.DataFrame(
            decomp_ic,columns=['IC{}_sv_0'format(num),'IC{}_sv_1'.format(num),'IC{}_sv_2'.format(num)]
        )
    else:
        svd_ic_df = pd.DataFrame(
            decomp_ic,columns=['IC{}_sv_0'.format(num),'IC{}_sv_1'.format(num),'IC{}_sv_2'.format(num)]
        )
        svd_ic_dfs = pd.concat([svd_ic_dfs,svd_ic_df], axis=1)
svd_ic_dfs.to_csv('image_svd_features.csv')

CPU times: user 1h 31min 44s, sys: 2min 15s, total: 1h 34min
Wall time: 1h 36min 1s
